<a href="https://colab.research.google.com/github/jmorala/TFMDS/blob/main/cuadernos/01_exploracion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicializar directorios
Clonar repositorio github
Posicionarse en el directorio raíz

In [1]:
# Este comando descarga el repositorio entero a una carpeta llamada 'TFMDS' en Colab.
!git clone https://github.com/jmorala/TFMDS.git

Cloning into 'TFMDS'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 50 (delta 19), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 4.17 MiB | 3.48 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [78]:
import os

# Detectar si estamos en Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Configurar el directorio de trabajo según el entorno
if IN_COLAB:
    os.chdir('TFMDS')
else:
    # En VS Code local, nos movemos al directorio raíz del proyecto
    # Usa raw string para evitar errores de escape en rutas Windows
    current_dir = r'C:\Users\jmora\Documents\TFMDS'
    os.chdir(current_dir)

# OPCIONAL: Para verificar que estás en la ruta correcta y ver las carpetas
print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: C:\Users\jmora\Documents\TFMDS


# Compresión inicial y definición del problema
## Generación de los datos
Los datos han sido recibidos por el tutor de TFM

Se trata de un caso real de un punto de venta de una empresa de distribución de productos para el automóvil

La información que se proporciona en los ficheros es de 1000 productos anonimizados por motivos de confidencialidad

## Objetivo
El objetivo es generar modelos que prevean las ventas en un horizonte de tiempo (***pendiente definir este horizonte***), y elegir el que dé mejor resultado.

Para el desarrollo del modelo hay que valorar si conviene un modelo para todos los productos o uno para cada cluster de productos.

Una vez seleccionado el modelo con mejor precisión, se estimará el stock final necesario.

Después se comparará con un modelo Naive para ver qué modelo tiene mejor resultado

## Modelo de datos
Los datos recibidos están en una hoja de cálculo con varias pestañas, cada pestaña ha sido exportada a un fichero csv con separado de campos el caracter ';'.

Cada fichero contiene los siguientes datos:


* Ventas.csv: Histórico de ventas de los 1000 productos. Contiene los campos:
producto: código de producto (entero)
idSecuencia: fecha del día de venta (AAAAMMDD)
udsVenta: Unidades vendidas ese día (entero)
* Calendario.csv: Indica la fecha de festivo y fecha de apertura del punto de venta. Contiene los campos: idSecuencia: fecha del día de venta (AAAAMMDD) bolOpen: 1 si ese día la tienda estaba abierta 0 en el caso contrario. bolHoliday: 1 si ese día era festivo 0 en caso contrario.
* Promociones.csv: Contiene las fecha de inicio y fin de las campañas promocionales. Contiene llos campos: producto: código de producto. idSecuenciaIni: fecha inicial de promoción para el producto (AAAAMMDD). idSecuenciaFin: Último día de propomoción para el producto (AAAAMMDD)
* Stock.csv: No será una variable exógena para el modelo, pero sirve para identificar días con ventas cero por rotura de stock (stock=0), En estos casos, conviene, “reconstruir” las ventas para que los modelos no aprendan de estos periodos excepcionales de ventas a cero debido a roturas. Contiene los campos: producto: Código del producto. idSecuencia: día referente del stock (AAAAMMDD). udsStock: Unidades en el stock para ese día (entero).

***Incluir dibujo del modelo de datos***

# Tratamiento y Preprocesamiento de los datos

Para cada juego de datos:

*   Convertir los datos al tipo adecuado
*   No hay datos faltantes o nulos. En cuyo caso ver cómo actuar
*   En la serie temporal no faltan días
*   Comprobar que los conjuntos de datos son coherentes en cuanto fechas y claves foráneas
*   Unir los datos en un único dataframe



## Archivo ventas.csv
acciones:
* Convertir los tipos de datos a los correctos
* Comprobar que no hay datos nulos
* Todos los productos tienen las mismas fechas informadas
* Están informados todos los días en el mismo rango de fechas para todos los artículos

In [79]:
import pandas as pd

# Ruta relativa del archivo CSV
RUTA_DATOS = 'datos/Ventas.csv'

# Cargar el archivo en un DataFrame de Pandas
dfventas = pd.read_csv(RUTA_DATOS, sep=';',
    parse_dates=['idSecuencia'],
    date_format='%Y%m%d')

# Muestra las primeras filas y la información de las columnas para iniciar la exploración
print("Primeras filas del DataFrame:")
print(dfventas.head())

print("\nInformación de las columnas y tipos de datos:")
dfventas.info()

Primeras filas del DataFrame:
   producto idSecuencia  udsVenta
0         1  2022-11-05        40
1         1  2022-11-06         0
2         1  2022-11-07        12
3         1  2022-11-08        28
4         1  2022-11-09        14

Información de las columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654408 entries, 0 to 654407
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   producto     654408 non-null  int64         
 1   idSecuencia  654408 non-null  datetime64[ns]
 2   udsVenta     654408 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 15.0 MB


In [80]:
prod_vc = dfventas['producto'].value_counts()
print(prod_vc)
print(f'lecturas diferentes: {prod_vc.nunique()}')

producto
1       732
2       732
3       732
4       732
5       732
       ... 
996     732
997     732
998     732
999     732
1000    732
Name: count, Length: 894, dtype: int64
lecturas diferentes: 1


El número de productos es 894 y tienen 732 lecturas cada uno, no hay nulos en los datos

In [81]:
# Comprobar que todos los artículos tienen información de todos los días y coinciden en el rango de fechas

fecha_min = dfventas['idSecuencia'].min()
fecha_max = dfventas['idSecuencia'].max()
dias_esperados = (fecha_max - fecha_min).days + 1

resumen = dfventas.groupby('producto').agg({
    'idSecuencia': ['min', 'max', 'nunique']
})

validacion_ok = (
    (resumen[('idSecuencia', 'min')] == fecha_min).all() and
    (resumen[('idSecuencia', 'max')] == fecha_max).all() and
    (resumen[('idSecuencia', 'nunique')] == dias_esperados).all()
)

print(f"¿Datos completos y consistentes? {validacion_ok}")
print(f"Rango de fechas: {fecha_min} a {fecha_max}")
print(f"Número de días en cada artículo: {dias_esperados}")
print(resumen)



¿Datos completos y consistentes? True
Rango de fechas: 2022-11-05 00:00:00 a 2024-11-05 00:00:00
Número de días en cada artículo: 732
         idSecuencia                   
                 min        max nunique
producto                               
1         2022-11-05 2024-11-05     732
2         2022-11-05 2024-11-05     732
3         2022-11-05 2024-11-05     732
4         2022-11-05 2024-11-05     732
5         2022-11-05 2024-11-05     732
...              ...        ...     ...
996       2022-11-05 2024-11-05     732
997       2022-11-05 2024-11-05     732
998       2022-11-05 2024-11-05     732
999       2022-11-05 2024-11-05     732
1000      2022-11-05 2024-11-05     732

[894 rows x 3 columns]


Todos los productos tienen el mismo rango de fechas entre el 2022-11-05 hasta el 2024-11-05 y todos tienen todos los días informados

## Archivo Calendario.
acciones:
* Convertir los tipos de datos a los correctos
* Comprobar que no hay datos nulos
* Comprobar rango de fechas y que todos los días están informados
* El rango de fechas con las ventas debe coincidir, en caso contrario actualizar dataframes

In [82]:
# Ruta relativa del archivo CSV
RUTA_DATOS = 'datos/Calendario.csv'

# Cargar el archivo en un DataFrame de Pandas
dfcalendario = pd.read_csv(RUTA_DATOS, sep=';',
    parse_dates=['idSecuencia'],
    date_format='%Y%m%d')

# Muestra las primeras filas y la información de las columnas para iniciar la exploración
print("Primeras filas del DataFrame:")
print(dfcalendario.head())

print("\nInformación de las columnas y tipos de datos:")
dfcalendario.info()

Primeras filas del DataFrame:
  idSecuencia  bolOpen  bolHoliday
0  2025-02-05        1           0
1  2025-02-06        1           0
2  2023-12-23        1           0
3  2023-12-24        1           1
4  2024-09-02        1           0

Información de las columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   idSecuencia  837 non-null    datetime64[ns]
 1   bolOpen      837 non-null    int64         
 2   bolHoliday   837 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 19.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   idSecuencia  837 non-null    datetime64[ns]
 1   bolOpen      837 non-null    int64         
 2   b

Hay 837 días y no hay valores nulos

In [83]:
# Comprobar rango de fechas y que todos los días están informados

rango = dfcalendario['idSecuencia']
completo = (rango.max() - rango.min()).days + 1 == rango.nunique() == len(dfcalendario)
binario_ok = (
    dfcalendario['bolOpen'].isin([0, 1]).all() and
    dfcalendario['bolHoliday'].isin([0, 1]).all()
)

print(f"Calendario completo: {completo}")
print(f"Valores binarios OK: {binario_ok}")
print(f"Rango: {rango.min().date()} - {rango.max().date()} ({rango.nunique()} días)")
print(f"✅ TODO OK" if completo and binario_ok else "⚠️ HAY PROBLEMAS")

Calendario completo: True
Valores binarios OK: True
Rango: 2022-11-06 - 2025-02-19 (837 días)
✅ TODO OK


El rango de fechas va desde 2022-11-06 a 2025-02-19 y todos los días están informados y los datos de bolOpen y bolHoliday son correctos.

El rango de fechas no coincide con el de ventas (2022-11-05 - 2024-11-05), el día 2022-11-05 de ventas no tiene información de calendario, y a partir del día 2024-11-06 los datos de calendario no son relevantes al no tener información en ventas

Borrar fila de dfventas con fecha = fecha_min ya que no está en calendario, y fecha_min = fecha_min+ 1

In [84]:
# Borrar de dfventas la fila con idSecuencia = fecha_min ya que no está en calendario, y actualizar fecha_min
dfventas = dfventas[dfventas['idSecuencia'] != fecha_min]
# Añadir un día a fecha_min ya que se utiliza posteriormente
fecha_min += pd.Timedelta(days=1)
# Borrar filas de dfcalendario fuera del rango de fechas de dfventas
dfcalendario = dfcalendario[
    (dfcalendario['idSecuencia'] >= fecha_min) &
    (dfcalendario['idSecuencia'] <= fecha_max)
]

## Archivo Promociones.csv
Acciones:
* Comprobar que no hay nulos
* Todos los producto están en ventas
* Eliminar rangos de fechas que no están en ventas
* Adecuar los rangos de fechas que se solapan a las fechas de ventas


In [85]:
# Ruta relativa del archivo CSV
RUTA_DATOS = 'datos/Promociones.csv'

# Cargar el archivo en un DataFrame de Pandas
dfpromociones = pd.read_csv(RUTA_DATOS, sep=';',
    parse_dates=['idSecuenciaIni', 'idSecuenciaFin'],
    date_format='%Y%m%d')

# Muestra las primeras filas y la información de las columnas para iniciar la exploración
print("Primeras filas del DataFrame:")
print(dfpromociones.head())

print("\nInformación de las columnas y tipos de datos:")
dfpromociones.info()

Primeras filas del DataFrame:
   producto idSecuenciaIni idSecuenciaFin
0       469     2012-09-27     2012-11-21
1       554     2015-10-09     2015-11-17
2       972     2013-01-12     2013-03-11
3       117     2022-08-11     2022-09-07
4       399     2021-05-06     2021-06-06

Información de las columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24718 entries, 0 to 24717
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   producto        24718 non-null  int64         
 1   idSecuenciaIni  24718 non-null  datetime64[ns]
 2   idSecuenciaFin  24718 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 579.5 KB


Hay 24.718 promociones y no hay valores nulos

In [86]:
# Verificar productos que no existen en dfventas
productos_invalidos = ~dfpromociones['producto'].isin(dfventas['producto'])
filas_productos_invalidos = dfpromociones[productos_invalidos]

# Verificar fechas fuera de rango
fechas_fuera_rango = (
    (dfpromociones['idSecuenciaIni'] < fecha_min) |
    (dfpromociones['idSecuenciaFin'] > fecha_max)
)
filas_fechas_invalidas = dfpromociones[fechas_fuera_rango]

# Mostrar resultados
print(f"Productos no existentes en dfventas: {productos_invalidos.sum()}")
if productos_invalidos.sum() > 0:
    print(filas_productos_invalidos)

print(f"\nFechas fuera de rango [{fecha_min.date()} - {fecha_max.date()}]: {fechas_fuera_rango.sum()}")
if fechas_fuera_rango.sum() > 0:
    print(filas_fechas_invalidas)

# Resumen
print(f"\n{'='*50}")
print(f"Total filas con problemas: {(productos_invalidos | fechas_fuera_rango).sum()}")

Productos no existentes en dfventas: 0

Fechas fuera de rango [2022-11-06 - 2024-11-05]: 21867
       producto idSecuenciaIni idSecuenciaFin
0           469     2012-09-27     2012-11-21
1           554     2015-10-09     2015-11-17
2           972     2013-01-12     2013-03-11
3           117     2022-08-11     2022-09-07
4           399     2021-05-06     2021-06-06
...         ...            ...            ...
24713       502     2013-01-12     2013-03-11
24714       423     2022-07-14     2022-08-10
24715        24     2017-01-10     2017-03-06
24716       647     2017-10-05     2017-11-05
24717       879     2019-03-29     2019-04-25

[21867 rows x 3 columns]

Total filas con problemas: 21867


Todos los productos en Promociones están en Ventas.

Hay muchas promociones que no pertenecen al rango de los datos de Ventas, se borran los datos en que idSecunciaFin es menor que fecha_min, idSecuenciaIni es mayor que fecha_max y los rangos que se solapan se fija IdSecuenciaIni o IdSecuenciaFin a fecha_min o fecha_nax

In [87]:
# Filtrar y ajustar promociones según el rango de ventas

# 1. Eliminar promociones completamente fuera de rango
# - Promociones que terminan antes de fecha_min
# - Promociones que empiezan después de fecha_max
dfpromociones = dfpromociones[
    ~((dfpromociones['idSecuenciaFin'] < fecha_min) |
      (dfpromociones['idSecuenciaIni'] > fecha_max))
].copy()

print(f"Filas después de eliminar promociones fuera de rango: {len(dfpromociones)}")

# 2. Ajustar fechas de promociones que se solapan con los límites
# Si idSecuenciaIni < fecha_min, ajustar a fecha_min
dfpromociones.loc[dfpromociones['idSecuenciaIni'] < fecha_min, 'idSecuenciaIni'] = fecha_min

# Si idSecuenciaFin > fecha_max, ajustar a fecha_max
dfpromociones.loc[dfpromociones['idSecuenciaFin'] > fecha_max, 'idSecuenciaFin'] = fecha_max

print(f"Fechas ajustadas al rango: [{fecha_min.date()} - {fecha_max.date()}]")

# 3. Verificación final
print(f"\nVerificación:")
print(f"- Todas las fechas ini >= fecha_min: {(dfpromociones['idSecuenciaIni'] >= fecha_min).all()}")
print(f"- Todas las fechas fin <= fecha_max: {(dfpromociones['idSecuenciaFin'] <= fecha_max).all()}")
print(f"- Total promociones válidas: {len(dfpromociones)}")

Filas después de eliminar promociones fuera de rango: 3096
Fechas ajustadas al rango: [2022-11-06 - 2024-11-05]

Verificación:
- Todas las fechas ini >= fecha_min: True
- Todas las fechas fin <= fecha_max: True
- Total promociones válidas: 3096


## Archivo Stock.csv
Acciones:
* Comprobar que no hay nulos
* Comprobar que todos los productos están en ventas, y que todos los productos de ventas están en stock
* Comprobar que las fechas-producto de ventas están en stock

In [88]:
# Ruta relativa del archivo CSV
RUTA_DATOS = 'datos/Stock.csv'

# Cargar el archivo en un DataFrame de Pandas
dfstock = pd.read_csv(RUTA_DATOS, sep=';',
    parse_dates=['idSecuencia'],
    date_format='%Y%m%d')

# Muestra las primeras filas y la información de las columnas para iniciar la exploración
print("Primeras filas del DataFrame:")
print(dfstock.head())

print("\nInformación de las columnas y tipos de datos:")
dfstock.info()

Primeras filas del DataFrame:
   producto idSecuencia  udsStock
0       240  2024-01-03        71
1       240  2024-01-04        71
2       240  2024-01-05        71
3       240  2024-01-06        68
4       240  2024-01-07        68

Información de las columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654408 entries, 0 to 654407
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   producto     654408 non-null  int64         
 1   idSecuencia  654408 non-null  datetime64[ns]
 2   udsStock     654408 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 15.0 MB


No hay valores nulos y tiene 654.408 datos

Comprobar que:
* Todos los productos de Stock están en Ventas
* Todos los producto de ventas están en Stock
* Todas las filas de Ventas con producto-Fecha (producto - idSecuencia) están en Stock

In [89]:
# Comprobaciones entre dfventas y dfstock

# 1) Productos: stock \ ventas y ventas \ stock
prod_stock = set(dfstock['producto'].unique())
prod_ventas = set(dfventas['producto'].unique())

prod_only_in_stock = sorted(prod_stock - prod_ventas)
prod_only_in_ventas = sorted(prod_ventas - prod_stock)

print(f"Productos en Stock pero NO en Ventas: {len(prod_only_in_stock)}")
if prod_only_in_stock:
    print("Ejemplos:", prod_only_in_stock[:10])

print(f"Productos en Ventas pero NO en Stock: {len(prod_only_in_ventas)}")
if prod_only_in_ventas:
    print("Ejemplos:", prod_only_in_ventas[:10])

# 2) Comprobar que todas las filas de Ventas (producto, idSecuencia) están en Stock
# Usamos merge con indicador para detectar filas de dfventas que no tienen match en dfstock
merge_pairs = dfventas.merge(
    dfstock[['producto', 'idSecuencia']],
    on=['producto', 'idSecuencia'],
    how='left',
    indicator=True
)

missing_pairs = merge_pairs[merge_pairs['_merge'] == 'left_only']
n_missing_pairs = len(missing_pairs)

print(f"\nFilas de Ventas sin correspondencia en Stock (producto, fecha): {n_missing_pairs} / {len(dfventas)}")
if n_missing_pairs:
    print("Primeras filas de ejemplo (Ventas sin Stock):")
    display(missing_pairs.head(10)[['producto', 'idSecuencia', 'udsVenta']])

# Resumen final booleano
all_stock_products_in_ventas = len(prod_only_in_stock) == 0
all_ventas_products_in_stock = len(prod_only_in_ventas) == 0
all_ventas_pairs_in_stock = n_missing_pairs == 0

print("\nResumen de validación:")
print(f"- Todos los productos de Stock están en Ventas? {all_stock_products_in_ventas}")
print(f"- Todos los productos de Ventas están en Stock? {all_ventas_products_in_stock}")
print(f"- Todas las filas (producto, fecha) de Ventas están en Stock? {all_ventas_pairs_in_stock}")

Productos en Stock pero NO en Ventas: 0
Productos en Ventas pero NO en Stock: 0

Filas de Ventas sin correspondencia en Stock (producto, fecha): 0 / 653514

Resumen de validación:
- Todos los productos de Stock están en Ventas? True
- Todos los productos de Ventas están en Stock? True
- Todas las filas (producto, fecha) de Ventas están en Stock? True

Filas de Ventas sin correspondencia en Stock (producto, fecha): 0 / 653514

Resumen de validación:
- Todos los productos de Stock están en Ventas? True
- Todos los productos de Ventas están en Stock? True
- Todas las filas (producto, fecha) de Ventas están en Stock? True


## Unir datos en un único dataframe

In [90]:
# Resultado: df_final con columnas: idSecuencia, producto, udsVenta, bolPromocion, bolOpen, bolHoliday
from collections import defaultdict

# Construir un diccionario producto -> lista de intervalos (inicio, fin)
promotions_dict = defaultdict(list)
for row in dfpromociones[['producto', 'idSecuenciaIni', 'idSecuenciaFin']].itertuples(index=False):
    prod, inicio, fin = row
    promotions_dict[prod].append((inicio, fin))

# Merge con calendario para obtener bolOpen y bolHoliday
df_merged = dfventas.merge(
    dfcalendario[['idSecuencia', 'bolOpen', 'bolHoliday']],
    on='idSecuencia',
    how='left'
)

# Función que determina si en una fecha dada hay promoción para ese producto
def _has_promo(prod, fecha):
    intervals = promotions_dict.get(prod)
    if not intervals:
        return 0
    for inicio, fin in intervals:
        if inicio <= fecha <= fin:
            return 1
    return 0

# Aplicar la función por filas
df_merged['bolPromocion'] = df_merged.apply(lambda r: _has_promo(r['producto'], r['idSecuencia']), axis=1).astype(int)

# Incorporar a df_merged el valor de udsStock, en que producto y idSecuencia coincidan
df_merged = df_merged.merge(
    dfstock[['producto', 'idSecuencia', 'udsStock']],
    on=['producto', 'idSecuencia'],
    how='left'
)

# Seleccionar y reordenar columnas finales
cols = ['idSecuencia', 'producto', 'udsVenta', 'bolPromocion', 'bolOpen', 'bolHoliday', 'udsStock']
existing_cols = [c for c in cols if c in df_merged.columns]
df_final = df_merged[existing_cols].copy()

# Verificaciones rápidas
print('df_final shape:', df_final.shape)
print('\nValores únicos en bolPromocion:', df_final['bolPromocion'].unique())
print('Filas con promoción (bolPromocion==1):', int(df_final['bolPromocion'].sum()))
print('Filas sin promoción (bolPromocion==0):', len(df_final) - int(df_final['bolPromocion'].sum()))
print('\nValores únicos en bolOpen:', df_final['bolOpen'].unique())
print('Valores con bolOpen == 1:', int(df_final['bolOpen'].sum()))
print('Valores con bolOpen == 0:', len(df_final) - int(df_final['bolOpen'].sum()))
print('\nValores únicos en bolHoliday:', df_final['bolHoliday'].unique())
print('Valores con bolHoliday == 1:', int(df_final['bolHoliday'].sum()))
print('Valores con bolHoliday == 0:', len(df_final) - int(df_final['bolHoliday'].sum()))
print('\nExisten valores nulos en df_final?:', df_final.isnull().any().any())

# Para cada producto contar cuantas filas tiene
prod_vc = df_final['producto'].value_counts()
print("\nNúmero de filas por producto:")
print(prod_vc)
print(f'lecturas diferentes: {prod_vc.nunique()}')

df_final.head()

df_final shape: (653514, 7)

Valores únicos en bolPromocion: [1 0]
Filas con promoción (bolPromocion==1): 85465
Filas sin promoción (bolPromocion==0): 568049

Valores únicos en bolOpen: [0 1]
Valores con bolOpen == 1: 544446
Valores con bolOpen == 0: 109068

Valores únicos en bolHoliday: [1 0]
Valores con bolHoliday == 1: 121584
Valores con bolHoliday == 0: 531930

Existen valores nulos en df_final?: False

Número de filas por producto:
producto
1       731
2       731
3       731
4       731
5       731
       ... 
996     731
997     731
998     731
999     731
1000    731
Name: count, Length: 894, dtype: int64
lecturas diferentes: 1


,idSecuencia,producto,udsVenta,bolPromocion,bolOpen,bolHoliday,udsStock
0,2022-11-06,1,0,1,0,1,148
1,2022-11-07,1,12,1,1,0,148
2,2022-11-08,1,28,1,1,0,136
3,2022-11-09,1,14,1,1,0,306
4,2022-11-10,1,26,1,1,0,291


# Análisis Exploratorio de Datos